In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
class Model(nn.Module):

    def __init__(self, in_features=4, h1=8, h2=9, out_features=3) -> None:
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

        

In [ ]:
torch.manual_seed(41)
model = Model()

In [ ]:
import matplotlib.pyplot as pl
import pandas as pd
%matplotlib inline

In [ ]:
url = 'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
df = pd.read_csv(url)


In [ ]:
df

In [ ]:
df['variety'] = df['variety'].replace('Setosa', 0.0)
df['variety'] = df['variety'].replace('Versicolor', 1.0)
df['variety'] = df['variety'].replace('Virginica', 2.0)




In [ ]:
features = df.drop('variety', axis=1)
output = df['variety']


In [ ]:
features = features.values
output = output.values

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(features, output, test_size=0.2, random_state=41)


In [ ]:
X_train = torch.FloatTensor(X_train)

X_test = torch.FloatTensor(X_test)

Y_train = torch.LongTensor(Y_train)

Y_test = torch.LongTensor(Y_test)



In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
import numpy
epochs = 100
losses = []

for i in range(epochs) :
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, Y_train)
    losses.append(loss.detach().numpy())
    if i%10 == 0 :
        print(f'epoch {i} loss = {loss}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()





In [ ]:
pl.plot(range(epochs), losses)
pl.xlabel('loss')
pl.ylabel('Epoch')

In [ ]:
with torch.no_grad():
    y_eval = model.forward(X_test)
    loss = criterion(y_eval, Y_test)

In [ ]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test): 
        y_val = model.forward(data)

        print(f'{i+1}.) {str(y_val)} \t {Y_test[i]},{y_val.argmax().item()}')

        if(y_val.argmax().item() == Y_test[i]):
            correct+=1
print(f'{correct}')
    